## Installing required libraries

In [ ]:
!pip install -qU ibm-watson-machine-learning
!pip install -qU langchain
!pip install -qU chromadb
!pip install -qU sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.5/405.5 kB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel

## Main

### Model loading from ibm watsonx

In [ ]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models import Model
import json

my_credentials = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : "INPUT_API_KEY_HERE"
}

model_id    = ModelTypes.FLAN_T5_XXL
gen_parms   = {GenParams.MAX_NEW_TOKENS: 200, GenParams.TOP_P: 0.6, GenParams.TOP_K: 3} # list of available parameters can be found at https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html#metanames.GenTextParamsMetaNames
project_id  = "INPUT_PROJECT_ID_HERE"
space_id    = None
verify      = False

model = Model(model_id, my_credentials, gen_parms, project_id, space_id, verify)

### Helper functions

In [ ]:
from langchain import PromptTemplate

def create_prompt(question, context=""):
  """
  Args
  ----------
  question: str
          question to the prompt template
  context: str, optional (default "")
          context to be provided to the question

  returns
  ----------
  prompt: str
        prompt created as input to llm via a prompt template with/without context

  """

  template = """
  Context: {context}
  Question: {question}

  Answer: """
  prompt_template = PromptTemplate(template=template, input_variables=['question', 'context'])
  prompt = prompt_template.format(question=question, context = context)
  return prompt

def generate_response(model, prompt, gen_params_override=None):
  """
  Args
  ----------
  model: modelobject
        IBM watsonx model to be used
  prompt: str
        prompt as input to the llm
  gen_params_override: optional (default = None)

  returns
  ----------
  output: tuple
        response from api (dict) and text generated (str)
  """
  generated_response = model.generate(prompt, gen_params_override)
  generated_text = model.generate_text(prompt, gen_params_override)
  # generated_text = generated_response['results'][0]['generated_text']
  output = (generated_response, generated_text)
  return output


def top_k_context(contexts, k=1):
  """
  Args
  ----------
  contexts: list
          top contexts returned by vector store
  k: int (default=1)
    top k context to be used

  returns
  ----------
  context: str
        context to be used in the prompt
  """
  context = ""
  for i in range(k):
    context += contexts[i].page_content
  return context

### Question answering based on Tennis

In [ ]:
# Question Answering without context
prompt = create_prompt("Who won the US open in 2019? How many sets were played?")

output = generate_response(model, prompt)
print(json.dumps(output[0], indent=2))

{
  "model_id": "google/flan-t5-xxl",
  "created_at": "2023-08-17T11:08:38.756Z",
  "results": [
    {
      "generated_text": "Roger Federer won the US Open in 2019 in straight sets.",
      "generated_token_count": 14,
      "input_token_count": 23,
      "stop_reason": "EOS_TOKEN"
    }
  ],
  "system": {
    "warnings": [
      {
        "message": "The model you are using is a Non-IBM Product. Review the Model information for details. URL: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx",
        "id": "DisclaimerWarning"
      }
    ]
  }
}


In [ ]:
# Question Answering with context
question = "Who won the US open in 2019? How many sets were played?"
context = "Rafael Nadal defeats Daniil Medvedev in five-set 2019 US Open final"

prompt = create_prompt(question, context)
output = generate_response(model, prompt)

print(json.dumps(output[0], indent=2))

{
  "model_id": "google/flan-t5-xxl",
  "created_at": "2023-08-17T11:08:41.085Z",
  "results": [
    {
      "generated_text": "Rafael Nadal defeats Daniil Medvedev in five-set 2019 US Open final",
      "generated_token_count": 21,
      "input_token_count": 43,
      "stop_reason": "EOS_TOKEN"
    }
  ],
  "system": {
    "warnings": [
      {
        "message": "The model you are using is a Non-IBM Product. Review the Model information for details. URL: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx",
        "id": "DisclaimerWarning"
      }
    ]
  }
}


#### Using langchain vector stores to provide context

- Great when use cases require loading of multiple documents
- Vector store is able to search the relevant content and return it as context for input to the LLM

In [ ]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

# Uncomment below line and input openai api key if using openai embeddings
# embeddings = OpenAIEmbeddings(openai_api_key="OPENAI_API_KEY_HERE")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [ ]:
raw_doc = TextLoader('/content/drive/MyDrive/IBM/wimbledon_19_final.txt').load() # edit the file path for your document
text_spliter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
documents = text_spliter.split_documents(raw_doc)
db = Chroma.from_documents(documents, embeddings)
mmr_retriever = db.as_retriever(search_type="mmr")

In [ ]:
query = "What is the fourth set summary in 100 words?"

# Uncomment and comment line accordingly to choose between similarity search or maximum marginal relevance search (MMR)
# context = db.similarity_search(query)
contexts = mmr_retriever.get_relevant_documents(query)

context = top_k_context(contexts, 2)
prompt = create_prompt(query, context)
output = generate_response(model, prompt)

print('Top context provided by vector store\n -------------------------------')
print(context)
print('\nLarge Language Model Response\n -------------------------------')
print(json.dumps(output[0], indent=2))

Top context provided by vector store
 -------------------------------
FOURTH SET
Federer, as he did regularly against Nadal in the semi-finals on Friday, began to attack anything shorty to rush Djokovic, who lost his serve at 2-2 when the Serbian hooked a backhand wide. Now stepping inside the baseline on backhand returns, Federer broke serve for a second time and a 5-2 advantage when Djokovic over-hit a backhand long. Djokovic regrouped and dug deep in the next game, breaking serve for the first time on his second break point chance, when a tired-looking Federer hit a slice backhand into the net. Perhaps, a hangover, as a result of saving the first break point with a backhand winner to end a 35-stroke rally. Djokovic soon closed the gap to 4-5, but Federer’s serve held up and he finished with a drive volley forehand winner — his 14th.A volley error by Federer on the third point of the tie-break handed Djokovic the early advantage and he held his nerve to join Sweden’s Bjorn Borg and G

In [ ]:
query = "What was the score of wimbledon 2019 final?"
# Uncomment and comment line accordingly to choose between similarity search or maximum marginal relevance search (MMR)
# context = db.similarity_search(query)
contexts = mmr_retriever.get_relevant_documents(query)

context = top_k_context(contexts, 2)
prompt = create_prompt(query, context)
output = generate_response(model, prompt)

print('Top context provided by vector store\n -------------------------------')
print(context)
print('\nLarge Language Model Response\n -------------------------------')
print(json.dumps(output[0], indent=2))

Top context provided by vector store
 -------------------------------
World No. 1 Novak Djokovic captured his fifth crown at The Championships, Wimbledon, on Sunday with a thrilling 7-6(5), 1-6, 7-6(4), 4-6, 13-12(3) victory over second seed Roger Federer, the eight-time former titlist from Switzerland, in four hours and 55 minutes on Centre Court. Federer had two championship points at 8-7, 40/15 on serve, in the fifth set that lasted two hours and two minutes.

It was the third major championship match that Djokovic saved two match points to beat Federer (also 2010 US Open semi-finals and 2011 US Open semi-finals). It was the first time since the 1948 Wimbledon final — American Robert Falkenburg beat John Bromwich of Australia 7-5, 0-6, 6-2, 3-6, 7-5, after being three match points down — that a player had been championship points down and won.FIRST SET
Federer made a lot of the early running, aggressive on his backhand return and using his slice to force Djokovic to hit up and come 

In [ ]:
query = "who won wimbledon 2019 final and the final score? is there any match point saved?"
# Uncomment and comment line accordingly to choose between similarity search or maximum marginal relevance search (MMR)
# context = db.similarity_search(query)
contexts = mmr_retriever.get_relevant_documents(query)

context = top_k_context(contexts, 2)
prompt = create_prompt(query, context)
output = generate_response(model, prompt)

print('Top context provided by vector store\n -------------------------------')
print(context)
print('\nLarge Language Model Response\n -------------------------------')
print(json.dumps(output[0], indent=2))

Top context provided by vector store
 -------------------------------
World No. 1 Novak Djokovic captured his fifth crown at The Championships, Wimbledon, on Sunday with a thrilling 7-6(5), 1-6, 7-6(4), 4-6, 13-12(3) victory over second seed Roger Federer, the eight-time former titlist from Switzerland, in four hours and 55 minutes on Centre Court. Federer had two championship points at 8-7, 40/15 on serve, in the fifth set that lasted two hours and two minutes.

It was the third major championship match that Djokovic saved two match points to beat Federer (also 2010 US Open semi-finals and 2011 US Open semi-finals). It was the first time since the 1948 Wimbledon final — American Robert Falkenburg beat John Bromwich of Australia 7-5, 0-6, 6-2, 3-6, 7-5, after being three match points down — that a player had been championship points down and won.FIRST SET
Federer made a lot of the early running, aggressive on his backhand return and using his slice to force Djokovic to hit up and come 